In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# MAIN PROGRAM: TF-IDF Cosine + pre-datataset

In [ ]:
#@title TF-IDF


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import string
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from collections import Counter
from typing import Dict, Text
from ast import literal_eval
from datetime import datetime
from wordcloud import WordCloud
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
stopwords_english = set(stopwords.words('english'))
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
from ast import literal_eval

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#data_02 = pd.read_csv("/content/gdrive/MyDrive/DBP_FALL2022/dbm_movie_EDA.tsv", sep="\t")

In [ ]:
#data_02 = pd.read_csv("/content/gdrive/MyDrive/DBP_FALL2022/dbm_movies_wo_ratings.tsv", sep="\t")

In [ ]:
data_02 = pd.read_csv("/content/gdrive/MyDrive/DBP_FALL2022/dbm_movies_final.tsv", sep="\t")

In [ ]:
data_02

,adult,budget,genres,id,original_language,original_title,overview,popularity,production_companies,production_countries,...,vote_average,vote_count,keywords,crew,year,month,day,day_name,characters,actors
0,False,30000000,"Animation, Comedy, Family",862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.950000,Pixar Animation Studios,United States of America,...,7.7,5416.0,"jealousy, toy, boy, friendship, friends, rival...","John Lasseter, Joss Whedon, Andrew Stanton, Jo...",1995,10,30,Monday,"Woody (voice), Buzz Lightyear (voice), Mr. Pot...","Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."
1,False,65000000,"Adventure, Fantasy, Family",8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.020000,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,...,6.9,2412.0,"board game, disappearance, based on children's...","Larry J. Franco, Jonathan Hensleigh, James Hor...",1995,12,15,Friday,"Alan Parrish, Samuel Alan Parrish / Van Pelt, ...","Robin Williams, Jonathan Hyde, Kirsten Dunst, ..."
2,False,0,"Romance, Comedy",15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.710000,"Warner Bros., Lancaster Gate",United States of America,...,6.5,92.0,"fishing, best friend, duringcreditsstinger, ol...","Howard Deutch, Mark Steven Johnson, Mark Steve...",1995,12,22,Friday,"Max Goldman, John Gustafson, Ariel Gustafson, ...","Walter Matthau, Jack Lemmon, Ann-Margret, Soph..."
3,False,16000000,"Comedy, Drama, Romance",31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.860000,Twentieth Century Fox Film Corporation,United States of America,...,6.1,34.0,"based on novel, interracial relationship, sing...","Forest Whitaker, Ronald Bass, Ronald Bass, Ezr...",1995,12,22,Friday,"Savannah 'Vannah' Jackson, Bernadine 'Bernie' ...","Whitney Houston, Angela Bassett, Loretta Devin..."
4,False,0,Comedy,11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.390000,"Sandollar Productions, Touchstone Pictures",United States of America,...,5.7,173.0,"baby, midlife crisis, confidence, aging, daugh...","Alan Silvestri, Elliot Davis, Nancy Meyers, Na...",1995,2,10,Friday,"George Banks, Nina Banks, Franck Eggelhoffer, ...","Steve Martin, Diane Keaton, Martin Short, Kimb..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43282,False,0,Horror,289923,en,The Burkittsville 7,A film archivist revisits the story of Rustin ...,0.386500,"Neptune Salad Entertainment, Pirie Productions",United States of America,...,7.0,1.0,"witch, mythology, legend, serial killer, mocku...","Ben Rock, Ben Rock",2000,10,3,Tuesday,"Branwall, Sarah Didonna, Kyle Brody, Bill Barn...","Monty Bane, Lucy Butler, David Grammer, Bill D..."
43283,False,0,Science Fiction,222848,en,Caged Heat 3000,It's the year 3000 AD. The world's most danger...,0.661600,Concorde-New Horizons,United States of America,...,3.5,1.0,NaN,"Roger Corman, Mike Elliott, Aaron Osborne, Mik...",1995,1,1,Sunday,"Kira (as Cassandra Leigh), Daly, Ruggs, Lewis,...","Lisa Boyle, Kena Land, Zaneta Polard, Don Yana..."
43284,False,0,Drama,111109,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178200,Sine Olivia,Philippines,...,9.0,3.0,"artist, play, pinoy","Lav Diaz, Lav Diaz, Dante Perez, Lav Diaz, Lav...",2011,11,17,Thursday,"Sister Angela, Homer, Crazy Woman/Virgin, Aman...","Angel Aquino, Perry Dizon, Hazel Orencio, Joel..."
43285,False,0,NaN,227506,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003504,Yermoliev,Russia,...,0.0,0.0,NaN,"Yakov Protazanov, Joseph N. Ermolieff",1917,10,21,Sunday,", , , ,","Iwan Mosschuchin, Nathalie Lissenko, Pavel Pav..."


In [ ]:
data_02['original_title'].head(10)

0                      Toy Story
1                        Jumanji
2               Grumpier Old Men
3              Waiting to Exhale
4    Father of the Bride Part II
5                           Heat
6                        Sabrina
7                   Tom and Huck
8                   Sudden Death
9                      GoldenEye
Name: original_title, dtype: object

In [ ]:
data_02['actors'].head(5)

0    Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...
1    Robin Williams, Jonathan Hyde, Kirsten Dunst, ...
2    Walter Matthau, Jack Lemmon, Ann-Margret, Soph...
3    Whitney Houston, Angela Bassett, Loretta Devin...
4    Steve Martin, Diane Keaton, Martin Short, Kimb...
Name: actors, dtype: object

In [ ]:
data_02['genres'].head(10)

0           Animation, Comedy, Family
1          Adventure, Fantasy, Family
2                     Romance, Comedy
3              Comedy, Drama, Romance
4                              Comedy
5      Action, Crime, Drama, Thriller
6                     Comedy, Romance
7    Action, Adventure, Drama, Family
8         Action, Adventure, Thriller
9         Adventure, Action, Thriller
Name: genres, dtype: object

In [ ]:
print("---- BEFORE ----")
print(data_02.isna().sum())
data_02.fillna('unknown', inplace=True)

print("---- AFTER ----")
print(data_02.isna().sum())

data_02

---- BEFORE ----
adult                       0
budget                      0
genres                   2287
id                          0
original_language           0
original_title              0
overview                    0
popularity                  0
production_companies    11291
production_countries     5921
revenue                     0
runtime                     0
spoken_languages         3687
tagline                     0
vote_average                0
vote_count                  0
keywords                13685
crew                      708
year                        0
month                       0
day                         0
day_name                    0
characters               2431
actors                   2284
dtype: int64
---- AFTER ----
adult                   0
budget                  0
genres                  0
id                      0
original_language       0
original_title          0
overview                0
popularity              0
production_companies    0


,adult,budget,genres,id,original_language,original_title,overview,popularity,production_companies,production_countries,...,vote_average,vote_count,keywords,crew,year,month,day,day_name,characters,actors
0,False,30000000,"Animation, Comedy, Family",862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.950000,Pixar Animation Studios,United States of America,...,7.7,5416.0,"jealousy, toy, boy, friendship, friends, rival...","John Lasseter, Joss Whedon, Andrew Stanton, Jo...",1995,10,30,Monday,"Woody (voice), Buzz Lightyear (voice), Mr. Pot...","Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."
1,False,65000000,"Adventure, Fantasy, Family",8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.020000,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,...,6.9,2412.0,"board game, disappearance, based on children's...","Larry J. Franco, Jonathan Hensleigh, James Hor...",1995,12,15,Friday,"Alan Parrish, Samuel Alan Parrish / Van Pelt, ...","Robin Williams, Jonathan Hyde, Kirsten Dunst, ..."
2,False,0,"Romance, Comedy",15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.710000,"Warner Bros., Lancaster Gate",United States of America,...,6.5,92.0,"fishing, best friend, duringcreditsstinger, ol...","Howard Deutch, Mark Steven Johnson, Mark Steve...",1995,12,22,Friday,"Max Goldman, John Gustafson, Ariel Gustafson, ...","Walter Matthau, Jack Lemmon, Ann-Margret, Soph..."
3,False,16000000,"Comedy, Drama, Romance",31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.860000,Twentieth Century Fox Film Corporation,United States of America,...,6.1,34.0,"based on novel, interracial relationship, sing...","Forest Whitaker, Ronald Bass, Ronald Bass, Ezr...",1995,12,22,Friday,"Savannah 'Vannah' Jackson, Bernadine 'Bernie' ...","Whitney Houston, Angela Bassett, Loretta Devin..."
4,False,0,Comedy,11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.390000,"Sandollar Productions, Touchstone Pictures",United States of America,...,5.7,173.0,"baby, midlife crisis, confidence, aging, daugh...","Alan Silvestri, Elliot Davis, Nancy Meyers, Na...",1995,2,10,Friday,"George Banks, Nina Banks, Franck Eggelhoffer, ...","Steve Martin, Diane Keaton, Martin Short, Kimb..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43282,False,0,Horror,289923,en,The Burkittsville 7,A film archivist revisits the story of Rustin ...,0.386500,"Neptune Salad Entertainment, Pirie Productions",United States of America,...,7.0,1.0,"witch, mythology, legend, serial killer, mocku...","Ben Rock, Ben Rock",2000,10,3,Tuesday,"Branwall, Sarah Didonna, Kyle Brody, Bill Barn...","Monty Bane, Lucy Butler, David Grammer, Bill D..."
43283,False,0,Science Fiction,222848,en,Caged Heat 3000,It's the year 3000 AD. The world's most danger...,0.661600,Concorde-New Horizons,United States of America,...,3.5,1.0,unknown,"Roger Corman, Mike Elliott, Aaron Osborne, Mik...",1995,1,1,Sunday,"Kira (as Cassandra Leigh), Daly, Ruggs, Lewis,...","Lisa Boyle, Kena Land, Zaneta Polard, Don Yana..."
43284,False,0,Drama,111109,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178200,Sine Olivia,Philippines,...,9.0,3.0,"artist, play, pinoy","Lav Diaz, Lav Diaz, Dante Perez, Lav Diaz, Lav...",2011,11,17,Thursday,"Sister Angela, Homer, Crazy Woman/Virgin, Aman...","Angel Aquino, Perry Dizon, Hazel Orencio, Joel..."
43285,False,0,unknown,227506,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003504,Yermoliev,Russia,...,0.0,0.0,unknown,"Yakov Protazanov, Joseph N. Ermolieff",1917,10,21,Sunday,", , , ,","Iwan Mosschuchin, Nathalie Lissenko, Pavel Pav..."


In [ ]:
def top_movies(num = 100, quantile = 0.95, data = data_02):
    req_votes = np.quantile(data['vote_count'], quantile)
    
    new_data = data[data['vote_count']>req_votes]
    means = new_data['vote_average']
    tot_mean = new_data['vote_average'].mean()
    votes = new_data['vote_count']
    
    new_data['ratings'] = (votes/(votes+req_votes)*means + req_votes/(votes+req_votes)*tot_mean)
    
    return new_data.sort_values('ratings', ascending = False).iloc[:num,:].reset_index(drop=True)

In [ ]:
def clean(x):
    res = []
    for w in x:
        if w not in stopwords_english:
            w = stemmer.stem(re.sub('[^a-zA-z0-9]', '', w))
            if(len(w)>0):
                res.append(w)
    return res

def preprocess(data_02):
    stemmer = PorterStemmer()
    docs = []
    DF = {}
    
    for ind, row in data_02.iterrows():
        set_title = set()
        doc_count = 0
        title = row['original_title'].lower().strip().split()
        title += row['genres'].lower().strip().split()
        title += row['actors'].lower().strip().split()
        body = row['overview'].lower().strip().split()
        body = clean(body)
        
        for word in title+body:
            if word in DF:
                if row.name in DF[word]:
                    DF[word][row.name]+=1
                else:
                    DF[word][row.name] = 1
            else:
                DF[word] = {}
                DF[word][row.name] = 1
            
            doc_count+=1
            if word in title:
                set_title.add(word)
        docs.append((doc_count, set_title))

    return DF, docs

In [ ]:
def generate_cosine_tfidf(data_02, alpha = 0.6):
    DF, docs = preprocess(data_02)
    tf_idf = np.zeros((len(docs), len(DF)))
    
    for i, (word, key) in enumerate(DF.items()):
        w_count = np.sum([value for _, value in key.items()])
        for ind, value in key.items():
            tf = value/docs[ind][0]
            idf = np.log(len(docs)/(w_count+1))
            tfidf = tf*idf*(1-alpha)
            if word in docs[ind][1]:
                tfidf = tf*idf*alpha
            
            tf_idf[ind][i] = tfidf
    return cosine_similarity(tf_idf)

In [ ]:
movies = top_movies(40000)
tf_idf = generate_cosine_tfidf(movies, alpha = 0.4)

In [ ]:
movies.head(5)

,adult,budget,genres,id,original_language,original_title,overview,popularity,production_companies,production_countries,...,vote_count,keywords,crew,year,month,day,day_name,characters,actors,ratings
0,False,25000000,"Drama, Crime",278,en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,51.66,"Castle Rock Entertainment, Warner Bros.",United States of America,...,8360.0,"prison, corruption, police brutality, prison c...","Frank Darabont, Niki Marvin, Thomas Newman, Ro...",1994,9,23,Friday,"Andy Dufresne, Ellis Boyd 'Red' Redding, Warde...","Tim Robbins, Morgan Freeman, Bob Gunton, Clanc...",8.409865
1,False,6000000,"Drama, Crime",238,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",41.10,"Paramount Pictures, Alfran Productions",United States of America,...,6024.0,"italy, love at first sight, loss of father, pa...","Francis Ford Coppola, Francis Ford Coppola, Gr...",1972,3,14,Tuesday,"Don Vito Corleone, Michael Corleone, Santino '...","Marlon Brando, Al Pacino, James Caan, Richard ...",8.377204
2,False,185000000,"Drama, Action, Crime, Thriller",155,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,123.20,"DC Comics, Legendary Pictures, Warner Bros., D...","United Kingdom, United States of America",...,12270.0,"dc comics, crime fighter, secret identity, sca...","Lucinda Syson, Hans Zimmer, Charles Roven, Chr...",2008,7,16,Wednesday,"Bruce Wayne / Batman, Alfred Pennyworth, Joker...","Christian Bale, Michael Caine, Heath Ledger, A...",8.244292
3,False,63000000,Drama,550,en,Fight Club,A ticking-time-bomb insomniac and a slippery s...,63.88,"Twentieth Century Fox Film Corporation, Regenc...","Germany, United States of America",...,9680.0,"support group, dual identity, nihilism, rage a...","Arnon Milchan, Michael Kaplan, Art Linson, Ale...",1999,10,15,Friday,"The Narrator, Tyler Durden, Robert 'Bob' Pauls...","Edward Norton, Brad Pitt, Meat Loaf, Jared Let...",8.230010
4,False,8000000,"Thriller, Crime",680,en,Pulp Fiction,"A burger-loving hit man, his philosophical par...",141.00,"Miramax Films, A Band Apart, Jersey Films",United States of America,...,8670.0,"transporter, brothel, drug dealer, boxer, mass...","Quentin Tarantino, Quentin Tarantino, Roger Av...",1994,9,10,Saturday,"Vincent Vega, Jules Winfield, Mia Wallace, But...","John Travolta, Samuel L. Jackson, Uma Thurman,...",8.222235


In [ ]:
Title = []
Genres = []
Actors = []
Overview = []

In [ ]:
Title_02 = []
Genres_02 = []
Actors_02 = []
Overview_02 = []

In [ ]:
Title_03 = []
Genres_03 = []
Actors_03 = []
Overview_03 = []

In [ ]:
Cosine_SC_01 = []
Cosine_SC_02 = []
Cosine_SC_03 = []

In [ ]:
def predict_movies(movie_name = "The Dark Knight", num = 10, verbose = 2, out = True, data = movies, tf_idf = tf_idf):
    try:
        ind = data[data['original_title'] == movie_name].index[0]
        output = data.loc[[val for val in np.argsort(tf_idf[ind])[::-1][1:num+1]]]
        if verbose == 2:
            print("The TF-IDF Cosine similarity scores for relevant movies are as follows:\n")
            print([Cosine_SC_03.append(round(val,2)) for val in np.sort(tf_idf[ind])[::-1][1:num+1]], '\n')
        if verbose >= 1:
            print(f'The top {num} recommended movies for "{movie_name}" are as follows:\n')
            for ind, row in output.iterrows():
                print("Title: {}".format(row['original_title']))
                Title_03.append(row['original_title'])
                print("Rating: {}".format(round(row['ratings'], 1)))
                print("Genres: {}".format(row['genres']))
                Genres_03.append(row['genres'])
                print("Actors: {}".format(row['actors']))
                Actors_03.append(row['actors'])
                print("Overview: {}".format(row['overview']))
                Overview_03.append(row['overview'])
                #print("IMDB Link: https://www.imdb.com/title/{}".format(row['imdb_id']))
                print("*****************************************************\n")
        if(out):
            return output
    except:
        print("MOVIE NOT FOUND!")

In [ ]:
predict_movies("Toy Story", num = 10, verbose = 2, out = False, data = movies, tf_idf = tf_idf)

The TF-IDF Cosine similarity scores for relevant movies are as follows:

[None, None, None, None, None, None, None, None, None, None] 

The top 10 recommended movies for "Toy Story" are as follows:

Title: Toy Story 2
Rating: 7.2
Genres: Animation, Comedy, Family
Actors: Tom Hanks, Tim Allen, Joan Cusack, Kelsey Grammer, Don Rickles, Jim Varney, Wallace Shawn, John Ratzenberger, Annie Potts, Wayne Knight, John Morris, Laurie Metcalf, Estelle Harris, R. Lee Ermey, Jodi Benson, Jonathan Harris, Joe Ranft, Andrew Stanton, Jeff Pidgeon, Corey Burton, John Lasseter, Mickie McGowan, Phil Proctor, Lee Unkrich
Overview: Andy heads off to Cowboy Camp, leaving his toys to their own devices. Things shift into high gear when an obsessive toy collector named Al McWhiggen, owner of Al's Toy Barn kidnaps Woody. Andy's toys mount a daring rescue mission, Buzz Lightyear meets his match and Woody has to decide where he and his heart truly belong.
*****************************************************

Ti

In [ ]:
print(Cosine_SC_01)

[0.49, 0.41, 0.15, 0.13, 0.08, 0.08, 0.07, 0.07, 0.07, 0.07]


In [ ]:
predict_movies("Pinocchio", num = 10, verbose = 2, out = False, data = movies, tf_idf = tf_idf)

The TF-IDF Cosine similarity scores for relevant movies are as follows:

[None, None, None, None, None, None, None, None, None, None] 

The top 10 recommended movies for "Pinocchio" are as follows:

Title: Bedazzled
Rating: 6.1
Genres: Fantasy, Comedy, Romance
Actors: Elizabeth Hurley, Brendan Fraser, Frances O'Connor, Orlando Jones, Paul Adelstein, Toby Huss, Gabriel Casseus, Miriam Shor, Brian Doyle-Murray, Jeff Doucette, Aaron Lustig
Overview: Elliot Richardson, suicidal techno geek, is given seven wishes to turn his life around when he meets up with a very seductive Satan. The catch: his soul. Some of his wishes include a 7 foot basketball star, a rock star, and a hamburger. But, as could be expected, the Devil must put her own little twist on each his fantasies.
*****************************************************

Title: Liar Liar
Rating: 6.5
Genres: Comedy
Actors: Jim Carrey, Maura Tierney, Cary Elwes, Jennifer Tilly, Swoosie Kurtz, Sara Paxton, Anne Haney, Amanda Donohoe, Jaso

In [ ]:
predict_movies("Titanic", num = 10, verbose = 2, out = False, data = movies, tf_idf = tf_idf)

The TF-IDF Cosine similarity scores for relevant movies are as follows:

[None, None, None, None, None, None, None, None, None, None] 

The top 10 recommended movies for "Titanic" are as follows:

Title: La leggenda del pianista sull'oceano
Rating: 7.5
Genres: Fantasy
Actors: Tim Roth, Pruitt Taylor Vince, Mélanie Thierry, Bill Nunn, Gabriele Lavia, Clarence Williams III, Mélanie Thierry, Peter Vaughan, Niall O'Brien, Alberto Vazquez, Luca De Luigi, Femi Elufowoju Jr., Nigel Fan, Roger Monk, Leonid Zaslavski, Bernard Padden
Overview: The story of a virtuoso piano player who lives his entire life aboard an ocean liner. Born and raised on the ship, 1900 (Tim Roth) learned about the outside world through interactions with passengers, never setting foot on land, even for the love of his life. Years later, the ship may be destroyed, and a former band member fears that 1900 may still be aboard, willing to go down with the ship.
*****************************************************

Title: Se

In [ ]:
#@title Create DataFrame


In [ ]:
print(Title)
print(Title_02)
print(Title_03)

['Toy Story 2', 'Toy Story 3', 'The 40 Year Old Virgin', 'How to Lose a Guy in 10 Days', 'Man on the Moon', 'Match Point', 'Small Soldiers', 'Monsters, Inc.', 'The Thin Red Line', 'Step Up 2: The Streets']
['Bedazzled', 'Liar Liar', 'Eat Pray Love', 'Big', 'Chicken Little', 'Spaceballs', 'Ella Enchanted', 'Rush Hour 2', '17 Again', 'The Giver']
["La leggenda del pianista sull'oceano", 'Serenity', 'Ghost Ship', 'Master and Commander: The Far Side of the World', 'Wrath of the Titans', 'Event Horizon', 'Return of the Jedi', 'Triangle', 'Wreck-It Ralph', 'Any Given Sunday']


In [ ]:

# importing the modules
from IPython.display import display
import pandas as pd
  
# creating a DataFrame
dict = {'Title' : Title,
        'Genres' : Genres,
        'Actors' : Actors,
        'Overview' : Overview,
        'Cosine Similarity': Cosine_SC_01}
df1 = pd.DataFrame(dict)
  
# displaying the DataFrame
display(df1)

,Title,Genres,Actors,Overview,Cosine Similarity
0,Toy Story 2,"Animation, Comedy, Family","Tom Hanks, Tim Allen, Joan Cusack, Kelsey Gram...","Andy heads off to Cowboy Camp, leaving his toy...",0.49
1,Toy Story 3,"Animation, Family, Comedy","Tom Hanks, Tim Allen, Ned Beatty, Joan Cusack,...","Woody, Buzz, and the rest of Andy's toys haven...",0.41
2,The 40 Year Old Virgin,"Comedy, Romance","Steve Carell, Catherine Keener, Paul Rudd, Rom...",Andy Stitzer has a pleasant life with a nice a...,0.15
3,How to Lose a Guy in 10 Days,"Comedy, Romance","Kate Hudson, Matthew McConaughey, Kathryn Hahn...","An advice columnist, Andie Anderson (Kate Huds...",0.13
4,Man on the Moon,"Comedy, Drama, Romance","Jim Carrey, Courtney Love, Bob Zmuda, Danny De...",A film about the life and career of the eccent...,0.08
5,Match Point,"Drama, Thriller, Crime, Romance","Jonathan Rhys Meyers, Scarlett Johansson, Emil...",Match Point is Woody Allen’s satire of the Bri...,0.08
6,Small Soldiers,"Comedy, Adventure, Fantasy, Science Fiction, A...","Gregory Smith, Kirsten Dunst, Denis Leary, Phi...",When missile technology is used to enhance toy...,0.07
7,"Monsters, Inc.","Animation, Comedy, Family","John Goodman, Billy Crystal, Mary Gibbs, Steve...","James Sullivan and Mike Wazowski are monsters,...",0.07
8,The Thin Red Line,"Drama, History, War","Sean Penn, Adrien Brody, Jim Caviezel, Ben Cha...","Based on the graphic novel by James Jones, The...",0.07
9,Step Up 2: The Streets,"Music, Drama, Romance","Briana Evigan, Robert Hoffman, Will Kemp, Cass...",When rebellious street dancer Andie lands at t...,0.07


In [ ]:
# importing the modules
from IPython.display import display
import pandas as pd
  
# creating a DataFrame
dict = {'Title' : Title_02,
        'Genres' : Genres_02,
        'Actors' : Actors_02,
        'Overview' : Overview_02,
        'Cosine Similarity': Cosine_SC_02}
df2 = pd.DataFrame(dict)
  
# displaying the DataFrame
display(df2)

,Title,Genres,Actors,Overview,Cosine Similarity
0,Bedazzled,"Fantasy, Comedy, Romance","Elizabeth Hurley, Brendan Fraser, Frances O'Co...","Elliot Richardson, suicidal techno geek, is gi...",0.09
1,Liar Liar,Comedy,"Jim Carrey, Maura Tierney, Cary Elwes, Jennife...",Fletcher Reede is a fast-talking attorney and ...,0.07
2,Eat Pray Love,Drama,"Julia Roberts, James Franco, Javier Bardem, Bi...",Liz Gilbert had everything a modern woman is s...,0.06
3,Big,"Fantasy, Drama, Comedy, Romance, Family","Tom Hanks, Elizabeth Perkins, Robert Loggia, J...","A young boy, Josh Baskin makes a wish at a car...",0.06
4,Chicken Little,"Animation, Family, Comedy","Zach Braff, Joan Cusack, Dan Molina, Steve Zah...",When the sky really is falling and sanity has ...,0.06
5,Spaceballs,"Comedy, Science Fiction","Mel Brooks, Rick Moranis, Bill Pullman, Daphne...",When the nefarious Dark Helmet hatches a plan ...,0.06
6,Ella Enchanted,"Family, Fantasy, Comedy","Anne Hathaway, Hugh Dancy, Cary Elwes, Vivica ...",Ella lives in a magical world in which each ch...,0.05
7,Rush Hour 2,"Action, Comedy, Crime, Thriller","Chris Tucker, Jackie Chan, Zhang Ziyi, Roselyn...",It's vacation time for Carter as he finds hims...,0.05
8,17 Again,Comedy,"Zac Efron, Leslie Mann, Matthew Perry, Tyler S...","On the brink of a midlife crisis, 30-something...",0.05
9,The Giver,"Drama, Science Fiction","Jeff Bridges, Meryl Streep, Brenton Thwaites, ...","In a seemingly perfect community, without war,...",0.05


In [ ]:
# importing the modules
from IPython.display import display
import pandas as pd
  
# creating a DataFrame
dict = {'Title' : Title_03,
        'Genres' : Genres_03,
        'Actors' : Actors_03,
        'Overview' : Overview_03,
        'Cosine Similarity': Cosine_SC_03}
df3 = pd.DataFrame(dict)
  
# displaying the DataFrame
display(df3)

,Title,Genres,Actors,Overview,Cosine Similarity
0,La leggenda del pianista sull'oceano,Fantasy,"Tim Roth, Pruitt Taylor Vince, Mélanie Thierry...",The story of a virtuoso piano player who lives...,0.08
1,Serenity,"Science Fiction, Action, Adventure, Thriller","Nathan Fillion, Gina Torres, Alan Tudyk, Moren...",When the renegade crew of Serenity agrees to h...,0.07
2,Ghost Ship,"Horror, Mystery, Thriller","Gabriel Byrne, Julianna Margulies, Desmond Har...",After discovering a passenger ship missing sin...,0.06
3,Master and Commander: The Far Side of the World,Adventure,"Russell Crowe, Paul Bettany, James D'Arcy, Bil...",After an abrupt and violent encounter with a F...,0.06
4,Wrath of the Titans,Adventure,"Sam Worthington, Liam Neeson, Ralph Fiennes, E...",A decade after his heroic defeat of the monstr...,0.05
5,Event Horizon,"Horror, Science Fiction, Mystery","Laurence Fishburne, Sam Neill, Kathleen Quinla...",In the year 2047 a group of astronauts are sen...,0.05
6,Return of the Jedi,"Adventure, Action, Science Fiction","Mark Hamill, Harrison Ford, Carrie Fisher, Bil...",As Rebel leaders map their strategy for an all...,0.05
7,Triangle,Horror,"Melissa George, Liam Hemsworth, Emma Lung, Rac...",The story revolves around the passengers of a ...,0.05
8,Wreck-It Ralph,"Family, Animation, Comedy, Adventure","John C. Reilly, Sarah Silverman, Jack McBrayer...","Wreck-It Ralph is the 9-foot-tall, 643-pound v...",0.05
9,Any Given Sunday,Drama,"Al Pacino, Cameron Diaz, Dennis Quaid, James W...",A star quarterback gets knocked out of the gam...,0.05


In [ ]:
#@title Save DataFrame


In [ ]:
!pip install Dataframe_image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.6 MB 5.0 MB/s 


In [ ]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 4 not upgraded.
Need to get 91.7 MB of archives.
After this operation, 309 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 105.0.5195.102-0ubuntu0.18.04.1 [1,156 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 105.0.5195.102-0ubuntu0.1

In [ ]:
import dataframe_image as dfi

In [ ]:
dfi.export(df1, "df1.png")
dfi.export(df2, "df2.png")
dfi.export(df3, "df3.png")

In [ ]:
from PIL import Image

image_1 = Image.open(r'/content/df1.png')
im_1 = image_1.convert('RGB')
im_1.save(r'/content/TFIDF_ToyStory_new.pdf')

In [ ]:
image_2 = Image.open(r'/content/df2.png')
im_2 = image_2.convert('RGB')
im_2.save(r'/content/TFIDF_Pinocchio_new.pdf')

In [ ]:
image_3 = Image.open(r'/content/df3.png')
im_3 = image_3.convert('RGB')
im_3.save(r'/content/TFIDF_Titanic_new.pdf')